In [34]:
import pandas as pd

def load_data(ticker):

    filename = '../data/' + ticker + '_1min_firstratedata.csv'
    df = pd.read_csv(filename)
    df['ticker'] = ticker

    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df["date"] = df["timestamp"].dt.date
    df["time"] = df["timestamp"].dt.time
    return df

ticker = 'SPY'
df = load_data(ticker)

In [35]:
# Add the RSI indicator with period n to a df

def RSI(df, n):
    "function to calculate RSI"
    delta = df["close"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    df["up"] = up.round(4)
    df["down"] = down.round(4)
    AVG_Gain = df["up"].rolling(window=n).mean()
    AVG_Loss = abs(df["down"].rolling(window=n).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df["RSI_14"] = RSI.round(4)
    df = df.drop(columns=["up", "down"])

    return df

df = RSI(df, 14)

#df.head()

In [36]:
import datetime
import plotly.graph_objects as go


# Candlestick chart of one day
def chart_range(df, date):
    df = df[df["date"] == date]
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=df["timestamp"],
                open=df["open"],
                high=df["high"],
                low=df["low"],
                close=df["close"],
            )
        ]
    )
    fig.show()


# chart_range(df, datetime.date(2023, 5, 18))


# chart the RSI of one day
def chart_rsi(df, date):
    df = df[df["date"] == date]
    # only chart from 9:30 to 16:00
    df = df[df["time"] >= datetime.time(9, 30, 0)]
    df = df[df["time"] <= datetime.time(16, 0, 0)]
    fig = go.Figure(data=go.Scatter(x=df["timestamp"], y=df["RSI_14"]))
    # add horizontal line at 30 and 70
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=30,
        x1=df["timestamp"].max(),
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=70,
        x1=df["timestamp"].max(),
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.show()


# chart_rsi(df, datetime.date(2023, 5, 18))


def chart_rsi_with_trades(timestamps, rsi_values, enters, exits):
    fig = go.Figure(
        data=go.Scatter(x=timestamps, y=rsi_values, mode="lines", name="RSI")
    )
    fig.add_trace(
        go.Scatter(
            x=enters["timestamps"],
            y=enters["values"],
            mode="markers",
            marker=dict(color="green", size=8),
            name="Enter",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=exits["timestamps"],
            y=exits["values"],
            mode="markers",
            marker=dict(color="red", size=8),
            name="Exit",
        )
    )
    # Add horizontal lines at RSI 30 and 70
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=30,
        x1=timestamps[-1],
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=70,
        x1=timestamps[-1],
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )

    fig.show()

In [37]:
# Simulate one's day of RSI trading, returns a dataframe of trades

import datetime
import pandas as pd


def simulate_trade_RSI(date, entry, exit, df):
    # Filter data for the specified date and time range
    df_day = df[
        (df["date"] == date)
        & (df["time"] >= datetime.time(9, 30, 0))
        & (df["time"] <= datetime.time(16, 0, 0))
    ]

    # store trades in new empty dataframe
    trade_log = pd.DataFrame(
        columns=[
            "Date",
            "Direction",
            "Entry",
            "Exit",
            "TimeEnter",
            "TimeExit",
            "Gain%",
            "Gain$",
        ]
    )

    # Initialize variables
    trades = 0
    in_trade = False

    # Iterate through the dataframe
    for i in range(1, len(df_day)):
        if (
            not in_trade
            and df_day["RSI_14"].iloc[i - 1] < entry
            and df_day["RSI_14"].iloc[i] >= entry
        ):
            # Buy at entry condition

            entry_price = df_day["close"].iloc[i]
            in_trade = True
            timeEnter = df_day["time"].iloc[i - 1]

        elif (
            in_trade
            and df_day["RSI_14"].iloc[i - 1] > exit
            and df_day["RSI_14"].iloc[i] <= exit
        ):
            # Sell at exit condition
            exit_price = df_day["close"].iloc[i]

            # Calculate gain for the trade
            if entry_price != 0:
                trade_gain_percent = ((exit_price - entry_price) / entry_price) * 100
                trade_gain_percent = round(trade_gain_percent, 2)

                trade_gain_net = exit_price - entry_price
                trade_gain_net = round(trade_gain_net, 2)

                trades += 1

                # Append trade details to the trade log DataFrame
                trade_log.loc[len(trade_log)] = {
                    "Date": date,
                    "Direction": "Long",
                    "Entry": entry_price,
                    "Exit": exit_price,
                    "TimeEnter": timeEnter,
                    "TimeExit": df_day["time"].iloc[i],
                    "Gain%": trade_gain_percent,
                    "Gain$": trade_gain_net,
                }

                # Reset trade variables
                entry_price = 0
                exit_price = 0
                in_trade = False

    return trade_log


# simulate_trade_RSI(datetime.date(2023, 5, 18), 30, 70, df)

In [38]:
# Combine the simulated trades for a range of dates into a single dataframe

def simulate_date_range(start_date, end_date, entry, exit, df):
    day_datas = []

    for n in range((end_date - start_date).days + 1):
        date = start_date + datetime.timedelta(n)
        day_data = simulate_trade_RSI(date, entry, exit, df)
        if day_data is not None:
            day_datas.append(day_data)

    day_datas = pd.concat(day_datas, ignore_index=True)

    return day_datas

In [39]:
trades = simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), 30, 70, df)

# post-processing of the trades dataframe

def rsi_backtest_explain_results(trades):

    total_gain = trades["Gain$"].sum()
    total_trades = len(trades)

    average_percent_gain = round(trades["Gain%"].mean(), 2)

    print("Total trades: " + str(total_trades))
    print("Average percent gain per trade: " + str(average_percent_gain) + "%")
    print("Total percent gain: " + str(round(total_gain, 2)) + "%")

    # histogram of the percent gain per trade
    import plotly.express as px

    fig = px.histogram(trades, x="Gain%", nbins=100)
    fig.show()

/var/folders/hb/0dd7xx3d4mzgbx_w9m1fmb2h0000gn/T/ipykernel_9734/1167655356.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



In [43]:
# Full stack experiment

ticker = 'SPY'

df = load_data(ticker)
df = RSI(df, 14)

enter_exit_pairs_symetric = [(30, 70), (20, 80), (25, 75), (35, 65), (40, 60)]
enter_exit_pairs_asymetric = [(10, 30), (20, 40), (30, 50), (40, 60), (50, 70), (60, 80), (70, 90)]

all_pairs = enter_exit_pairs_symetric + enter_exit_pairs_asymetric

master = []
for e in all_pairs:
    master.append(simulate_date_range(datetime.date(2022, 9, 30), datetime.date(2023, 9, 30), e[0], e[1], df))
    print(e)
    rsi_backtest_explain_results(master[-1])

master = pd.concat(master, ignore_index=True)

(30, 70)
Total trades: 952
Average percent gain per trade: 0.02%
Total percent gain: 76.08%


/var/folders/hb/0dd7xx3d4mzgbx_w9m1fmb2h0000gn/T/ipykernel_9734/1167655356.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



(20, 80)
Total trades: 336
Average percent gain per trade: 0.09%
Total percent gain: 127.58%


/var/folders/hb/0dd7xx3d4mzgbx_w9m1fmb2h0000gn/T/ipykernel_9734/1167655356.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



(25, 75)
Total trades: 622
Average percent gain per trade: 0.03%
Total percent gain: 80.93%


...

/var/folders/hb/0dd7xx3d4mzgbx_w9m1fmb2h0000gn/T/ipykernel_9734/1167655356.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



(60, 80)
Total trades: 807
Average percent gain per trade: 0.07%
Total percent gain: 222.61%


/var/folders/hb/0dd7xx3d4mzgbx_w9m1fmb2h0000gn/T/ipykernel_9734/1167655356.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



(70, 90)
Total trades: 153
Average percent gain per trade: 0.28%
Total percent gain: 168.57%


/var/folders/hb/0dd7xx3d4mzgbx_w9m1fmb2h0000gn/T/ipykernel_9734/1167655356.py:12: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.

